# Exercises: CIFAR10 training

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

## Model

In [22]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)


model = keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(
        keras.layers.Dense(
            units=100,
            activation='elu',
            kernel_initializer='he_normal'
        )
    )
model.add(keras.layers.Dense(10, activation='softmax'))

## Data

In [23]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()
X_train_full.shape, y_train_full.shape

((50000, 32, 32, 3), (50000, 1))

## Train

In [24]:
from sklearn.model_selection import train_test_split

X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [25]:
def get_logs_dir():
    now = time.strftime('%d_%m_%Y-%H_%M_%H')
    return os.path.join('./logs', now)

gpu_name = tf.test.gpu_device_name()
logs_dir = get_logs_dir()
tensorboard_cb = keras.callbacks.TensorBoard(logs_dir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)

with tf.device(gpu_name):
    optimizer = keras.optimizers.Nadam(lr=5e-5)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer = optimizer,
        metrics='accuracy'
    )

    history_1 = model.fit(
        X_train,
        y_train,
        epochs=1000,
        validation_data=(X_valid, y_valid),
        callbacks=[
            tensorboard_cb,
            early_stopping_cb
        ]
    )

Epoch 1/1000
1407/1407 [==============================] - 15s 10ms/step - loss: 1.9227 - accuracy: 0.3027 - val_loss: 1.7982 - val_accuracy: 0.3474
Epoch 2/1000
1407/1407 [==============================] - 14s 10ms/step - loss: 1.7135 - accuracy: 0.3829 - val_loss: 1.7148 - val_accuracy: 0.3658
Epoch 3/1000
1407/1407 [==============================] - 15s 10ms/step - loss: 1.6279 - accuracy: 0.4134 - val_loss: 1.6666 - val_accuracy: 0.3836
Epoch 4/1000
1407/1407 [==============================] - 14s 10ms/step - loss: 1.5706 - accuracy: 0.4336 - val_loss: 1.6455 - val_accuracy: 0.4172
Epoch 5/1000
1407/1407 [==============================] - 14s 10ms/step - loss: 1.5266 - accuracy: 0.4516 - val_loss: 1.5641 - val_accuracy: 0.4318
Epoch 6/1000
1407/1407 [==============================] - 15s 10ms/step - loss: 1.4919 - accuracy: 0.4661 - val_loss: 1.5337 - val_accuracy: 0.4430
Epoch 7/1000
1407/1407 [==============================] - 15s 11ms/step - loss: 1.4597 - accuracy: 0.4761 - val_

In [28]:
model.save('model_1.h5')
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 3ms/step - loss: 1.5503 - accuracy: 0.5078


[1.55025053024292, 0.5077999830245972]

## Train with batch normalization

In [32]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
logs_dir = get_logs_dir()

# Model
model_bn = keras.models.Sequential()

model_bn.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model_bn.add(keras.layers.BatchNormalization())
for _ in range(20):
    model_bn.add(
        keras.layers.Dense(
            units=100,
            kernel_initializer='he_normal'
        )
    )
    model_bn.add(
        keras.layers.BatchNormalization()
    )
    model_bn.add(
        keras.layers.Activation('elu')
    )
model_bn.add(keras.layers.Dense(10, activation='softmax'))

# Training
tensorboard_cb = keras.callbacks.TensorBoard(logs_dir + '_bn_1')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("bn_model.h5", save_best_only=True)

with tf.device(gpu_name):
    optimizer = keras.optimizers.Nadam(lr=5e-4)
    model_bn.compile(
        loss='sparse_categorical_crossentropy',
        optimizer = optimizer,
        metrics='accuracy'
    )

    history_bn = model_bn.fit(
        X_train,
        y_train,
        epochs=1000,
        validation_data=(X_valid, y_valid),
        callbacks=[
            early_stopping_cb,
            model_checkpoint_cb,
            tensorboard_cb,
        ]
    )

model_bn.save('bn_model.h5')
print(model_bn.evaluate(X_valid, y_valid))

Epoch 1/1000
1407/1407 [==============================] - 43s 30ms/step - loss: 1.8377 - accuracy: 0.3421 - val_loss: 1.6416 - val_accuracy: 0.4134
Epoch 2/1000
1407/1407 [==============================] - 42s 30ms/step - loss: 1.6630 - accuracy: 0.4099 - val_loss: 1.6124 - val_accuracy: 0.4182
Epoch 3/1000
1407/1407 [==============================] - 42s 30ms/step - loss: 1.5956 - accuracy: 0.4321 - val_loss: 1.5340 - val_accuracy: 0.4410
Epoch 4/1000
1407/1407 [==============================] - 42s 30ms/step - loss: 1.5487 - accuracy: 0.4464 - val_loss: 1.4964 - val_accuracy: 0.4640
Epoch 5/1000
1407/1407 [==============================] - 42s 30ms/step - loss: 1.5011 - accuracy: 0.4648 - val_loss: 1.4506 - val_accuracy: 0.4836
Epoch 6/1000
1407/1407 [==============================] - 42s 30ms/step - loss: 1.4661 - accuracy: 0.4769 - val_loss: 1.4245 - val_accuracy: 0.4918
Epoch 7/1000
1407/1407 [==============================] - 43s 30ms/step - loss: 1.4349 - accuracy: 0.4892 - val_

## Train self-normalized network

In [35]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
logs_dir = get_logs_dir()

# Model
model_snn = keras.models.Sequential()

model_snn.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model_snn.add(
        keras.layers.Dense(
            units=100,
            kernel_initializer='lecun_normal',
            activation='selu'
        )
    )
model_snn.add(keras.layers.Dense(10, activation='softmax'))

# Training
tensorboard_cb = keras.callbacks.TensorBoard(logs_dir + '_snn_2')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("snn_model.h5", save_best_only=True)

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

with tf.device(gpu_name):
    optimizer = keras.optimizers.Nadam(lr=7e-4)
    model_snn.compile(
        loss='sparse_categorical_crossentropy',
        optimizer = optimizer,
        metrics='accuracy'
    )

    history_snn = model_snn.fit(
        X_train_scaled,
        y_train,
        epochs=1000,
        validation_data=(X_valid_scaled, y_valid),
        callbacks=[
            early_stopping_cb,
            model_checkpoint_cb,
            tensorboard_cb,
        ]
    )

model_snn.save('snn_model.h5')
print(model_snn.evaluate(X_valid_scaled, y_valid))

Epoch 1/1000
1407/1407 [==============================] - 15s 11ms/step - loss: 1.9258 - accuracy: 0.3081 - val_loss: 1.8344 - val_accuracy: 0.3380
Epoch 2/1000
1407/1407 [==============================] - 14s 10ms/step - loss: 1.7161 - accuracy: 0.3908 - val_loss: 1.6946 - val_accuracy: 0.3990
Epoch 3/1000
1407/1407 [==============================] - 14s 10ms/step - loss: 1.6147 - accuracy: 0.4301 - val_loss: 1.6928 - val_accuracy: 0.4004
Epoch 4/1000
1407/1407 [==============================] - 14s 10ms/step - loss: 1.5447 - accuracy: 0.4544 - val_loss: 1.6569 - val_accuracy: 0.4220
Epoch 5/1000
1407/1407 [==============================] - 14s 10ms/step - loss: 1.4904 - accuracy: 0.4764 - val_loss: 1.6517 - val_accuracy: 0.4404
Epoch 6/1000
1407/1407 [==============================] - 15s 10ms/step - loss: 1.4449 - accuracy: 0.4948 - val_loss: 1.5270 - val_accuracy: 0.4732
Epoch 7/1000
1407/1407 [==============================] - 15s 10ms/step - loss: 1.3956 - accuracy: 0.5090 - val_

In [38]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
logs_dir = get_logs_dir()

# Model
model_snn_da = keras.models.Sequential()

model_snn_da.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model_snn_da.add(
        keras.layers.Dense(
            units=100,
            kernel_initializer='lecun_normal',
            activation='selu'
        )
    )
model_snn_da.add(keras.layers.AlphaDropout(rate=0.1))
model_snn_da.add(keras.layers.Dense(10, activation='softmax'))

# Training
tensorboard_cb = keras.callbacks.TensorBoard(logs_dir + '_snn_da_1')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("snn_da_model.h5", save_best_only=True)

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

with tf.device(gpu_name):
    optimizer = keras.optimizers.Nadam(lr=5e-4)
    model_snn_da.compile(
        loss='sparse_categorical_crossentropy',
        optimizer = optimizer,
        metrics='accuracy'
    )

    history_snn_da = model_snn_da.fit(
        X_train_scaled,
        y_train,
        epochs=1000,
        validation_data=(X_valid_scaled, y_valid),
        callbacks=[
            early_stopping_cb,
            model_checkpoint_cb,
            tensorboard_cb,
        ]
    )

model_snn_da.save('snn_da_model.h5')
print(model_snn_da.evaluate(X_valid_scaled, y_valid))

Epoch 1/1000
1407/1407 [==============================] - 16s 11ms/step - loss: 1.8946 - accuracy: 0.3225 - val_loss: 1.7059 - val_accuracy: 0.3850
Epoch 2/1000
1407/1407 [==============================] - 15s 11ms/step - loss: 1.6681 - accuracy: 0.4082 - val_loss: 1.6394 - val_accuracy: 0.4072
Epoch 3/1000
1407/1407 [==============================] - 15s 11ms/step - loss: 1.5780 - accuracy: 0.4428 - val_loss: 1.6245 - val_accuracy: 0.4318
Epoch 4/1000
1407/1407 [==============================] - 15s 11ms/step - loss: 1.5025 - accuracy: 0.4716 - val_loss: 1.5461 - val_accuracy: 0.4546
Epoch 5/1000
1407/1407 [==============================] - 15s 11ms/step - loss: 1.4472 - accuracy: 0.4909 - val_loss: 1.5535 - val_accuracy: 0.4650
Epoch 6/1000
1407/1407 [==============================] - 15s 11ms/step - loss: 1.4039 - accuracy: 0.5098 - val_loss: 1.5309 - val_accuracy: 0.4830
Epoch 7/1000
1407/1407 [==============================] - 15s 11ms/step - loss: 1.3627 - accuracy: 0.5269 - val_

In [39]:
class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

model_snn_mcda = keras.models.Sequential([
    MCAlphaDropout(layer.rate)
    if isinstance(layer, keras.layers.AlphaDropout)
    else layer
    for layer in model_snn_da.layers
])

In [44]:
def mc_dropout_predict_probas(mc_model, X, n_samples=10):
    predictions = [mc_model.predict(X) for _ in range(n_samples)]
    return np.mean(predictions, axis=0)

def mc_dropout_predict_classes(mc_model, X, n_samples=10):
    probas = mc_dropout_predict_probas(mc_model, X, n_samples)
    return np.argmax(probas, axis=1)

In [45]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

y_pred = mc_dropout_predict_classes(model_snn_mcda, X_valid_scaled)
accuracy = np.mean(y_pred == y_valid[:, 0])
accuracy

0.5006